In [47]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import losses
from tensorflow.keras import metrics

import matplotlib.pyplot as plt
import numpy

In [32]:
data_generator = ImageDataGenerator(rescale=1.0/255., zoom_range=0.2, rotation_range=15,
                                             width_shift_range=0.05, height_shift_range=0.05)

In [33]:
DIRECTORY_TRAIN = "./Covid19-dataset/train"
DIRECTORY_TEST = "./Covid19-dataset/test"
CLASS_MODE = "categorical"
COLOR_MODE = "grayscale"
TARGET_SIZE = (256, 256)
BATCH_SIZE = 32

In [34]:
training_iterator = data_generator.flow_from_directory(DIRECTORY_TRAIN, class_mode=CLASS_MODE, color_mode=COLOR_MODE, 
                                                               target_size=TARGET_SIZE, batch_size=BATCH_SIZE)

Found 251 images belonging to 3 classes.


In [35]:
sample_batch_input, sample_batch_labels = training_iterator.next()

In [36]:
print(sample_batch_input.shape, sample_batch_labels.shape)

(32, 256, 256, 1) (32, 3)


In [37]:
test_iterator = data_generator.flow_from_directory(DIRECTORY_TEST, class_mode=CLASS_MODE, color_mode=COLOR_MODE, 
                                                               target_size=TARGET_SIZE, batch_size=BATCH_SIZE)

Found 66 images belonging to 3 classes.


In [38]:
sample_batch_input, sample_batch_labels = test_iterator.next()

In [39]:
print(sample_batch_input.shape, sample_batch_labels.shape)

(32, 256, 256, 1) (32, 3)


In [40]:
print(type(sample_batch_input), type(sample_batch_labels))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [49]:
def model_design(features):
    model = Sequential(name="classification_model")
    model.add(Inputs(shape=(256, 256, 1)))
    model.add(layers.Conv2D(2, 5, strides=3, padding="valid", activation="relu"))
    model.add(layers.MaxPooling2D(pool_size=(5, 5), strides=(5, 5)))
    model.add(layers.Conv2D(4, 3, strides=1, padding="valid", activation="relu"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))) 
    model.add(layers.Flatten())
    model.add(layers.Dense(3, activation="softmax"))
    opt = Adam(learning_rate=0.01)
    model.compile(optimizer=opt, loss=losses.CategoricalCrossEntropy() , metrics=[metrics.CategoricalAccuracy(), metrics.AUC()])
    return model